In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from pathlib import Path
import time
import random
import re
import os

In [4]:
class RusscoPDFDownloader:
    def __init__(self, base_url, output_folder="russco_guidelines"):
        self.base_url = base_url
        self.base_domain = urlparse(base_url).netloc
        self.output_folder = Path(output_folder)
        self.output_folder.mkdir(exist_ok=True)
        
        # Настройка сессии с заголовками браузера
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/pdf,*/*',
            'Accept-Language': 'ru-RU,ru;q=0.9,en;q=0.8',
            'Referer': base_url
        })
        
        print(f"📁 Папка для сохранения: {self.output_folder.absolute()}")

    def get_soup(self, url):
        """Получает и парсит HTML страницы."""
        try:
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            # Проверка кодировки для корректного отображения кириллицы
            response.encoding = response.apparent_encoding 
            return BeautifulSoup(response.text, 'lxml')
        except Exception as e:
            print(f"❌ Ошибка загрузки {url}: {e}")
            return None

    def find_disease_links(self, url=None):
        """
        Находит ссылки на страницы заболеваний на главной странице каталога.
        """
        url = url or self.base_url
        print(f"🔍 Сканирование страницы: {url}")
        
        soup = self.get_soup(url)
        if not soup:
            return []
        
        disease_links = []
        
        # Стратегия 1: Поиск ссылок, содержащих ключевые слова в тексте или URL
        # Обычно это теги <a> внутри списков или таблиц
        for link in soup.find_all('a', href=True):
            href = link['href']
            text = link.get_text(strip=True).lower()
            
            # Фильтр: ссылка должна вести на наш домен
            full_url = urljoin(url, href)
            if self.base_domain not in full_url:
                continue
            
            # Фильтр: исключаем служебные страницы
            skip_keywords = ['login', 'search', 'category', 'tag', 'author', 'mailto', '#', 'javascript']
            if any(kw in href.lower() for kw in skip_keywords):
                continue
            
            # Фильтр: текст ссылки должен быть длинным (название болезни), а не меню
            if len(text) > 15: 
                # Проверяем, не ведет ли ссылка сразу на PDF
                if href.lower().endswith('.pdf'):
                    disease_links.append({'title': text, 'url': full_url, 'type': 'direct_pdf'})
                else:
                    disease_links.append({'title': text, 'url': full_url, 'type': 'page'})
        
        # Удаление дубликатов
        seen = set()
        unique_links = []
        for item in disease_links:
            if item['url'] not in seen:
                seen.add(item['url'])
                unique_links.append(item)
        
        print(f"✅ Найдено потенциальных разделов: {len(unique_links)}")
        return unique_links

    def extract_pdf_from_page(self, page_url, disease_title):
        """
        Заходит на страницу заболевания и ищет ссылку на PDF.
        """
        soup = self.get_soup(page_url)
        if not soup:
            return None
        
        # Стратегия поиска PDF на внутренней странице
        pdf_urls = []
        
        # 1. Прямые ссылки на PDF
        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.lower().endswith('.pdf'):
                pdf_urls.append(urljoin(page_url, href))
        
        # 2. Кнопки или иконки со словом "Скачать" или "Рекомендации"
        for link in soup.find_all('a', href=True):
            text = link.get_text(strip=True).lower()
            if 'скачать' in text or 'рекомендации' in text or 'pdf' in text:
                href = link['href']
                # Если ссылка не явная, проверяем её содержимое HEAD-запросом
                if not href.endswith('.pdf'):
                    full_url = urljoin(page_url, href)
                    if self.is_pdf_url(full_url):
                        pdf_urls.append(full_url)
                else:
                    pdf_urls.append(urljoin(page_url, href))
        
        return list(set(pdf_urls))

    def is_pdf_url(self, url):
        """Проверяет заголовки URL, чтобы убедиться, что это PDF."""
        try:
            resp = self.session.head(url, timeout=10, allow_redirects=True)
            return 'application/pdf' in resp.headers.get('Content-Type', '')
        except:
            return False

    def download_file(self, url, filename):
        """Скачивает файл по ссылке."""
        try:
            # Проверка размера через HEAD
            head = self.session.head(url, timeout=10)
            size = int(head.headers.get('Content-Length', 0))
            if size > 50 * 1024 * 1024: # Лимит 50МБ
                print(f"⚠️ Пропущен (>{size//1024//1024}MB): {filename}")
                return False

            resp = self.session.get(url, stream=True, timeout=30)
            resp.raise_for_status()
            
            path = self.output_folder / filename
            with open(path, 'wb') as f:
                for chunk in resp.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            print(f"✅ Скачан: {filename}")
            return True
        except Exception as e:
            print(f"❌ Ошибка скачивания {filename}: {e}")
            return False

    def sanitize_filename(self, name):
        """Очищает название для имени файла."""
        name = re.sub(r'[<>:"/\\|?*]', '', name)
        name = name.replace(' ', '_').strip('_')
        return name[:100] # Ограничение длины

    def run(self):
        """Запускает процесс скачивания."""
        links = self.find_disease_links()
        
        if not links:
            print("⚠️ Ссылки не найдены. Возможно, сайт использует динамическую загрузку.")
            return

        total_downloaded = 0
        
        for i, item in enumerate(links, 1):
            print(f"\n[{i}/{len(links)}] Обработка: {item['title']}")
            
            # Если ссылка ведет сразу на PDF
            if item['type'] == 'direct_pdf':
                filename = f"{self.sanitize_filename(item['title'])}.pdf"
                if self.download_file(item['url'], filename):
                    total_downloaded += 1
                time.sleep(1)
                continue
            
            # Если ссылка ведет на страницу, ищем PDF там
            pdfs = self.extract_pdf_from_page(item['url'], item['title'])
            
            if pdfs:
                for pdf_url in pdfs:
                    # Генерируем имя файла
                    fname = os.path.basename(urlparse(pdf_url).path)
                    if not fname or fname == 'pdf':
                        fname = f"{self.sanitize_filename(item['title'])}.pdf"
                    
                    if self.download_file(pdf_url, fname):
                        total_downloaded += 1
            else:
                print(f"⚠️ PDF не найден для: {item['title']}")
            
            # Вежливая задержка
            # Генерируем случайное число секунд от 1 до 4
            random_sleep_time = random.uniform(1, 4)

            # Используем это случайное время в time.sleep()
            time.sleep(random_sleep_time)
        
        print(f"\n🎉 Готово! Скачано файлов: {total_downloaded}")
        print(f"📁 Путь: {self.output_folder.absolute()}")

# ==================== ЗАПУСК ====================
if __name__ == "__main__":
    # URL раздела с рекомендациями
    TARGET_URL = "https://www.rosoncoweb.ru/standarts/RUSSCO/"
    
    downloader = RusscoPDFDownloader(
        base_url=TARGET_URL,
        output_folder="russco_pdf_guidelines"
    )
    
    downloader.run()

📁 Папка для сохранения: /home/jupyter/notebooks/Champ_Sech_2026/russco_pdf_guidelines
🔍 Сканирование страницы: https://www.rosoncoweb.ru/standarts/RUSSCO/
✅ Найдено потенциальных разделов: 84

[1/84] Обработка: председатель - носов дмитрий александрович
✅ Скачан: molecular_genetic_research.pdf
✅ Скачан: председатель -_носов_дмитрий_александрович.pdf
✅ Скачан: temporary_recommendations.pdf

[2/84] Обработка: ветераны онкологии
✅ Скачан: molecular_genetic_research.pdf
✅ Скачан: ветераны_онкологии.pdf
⚠️ Пропущен (>114MB): veterans.pdf
✅ Скачан: temporary_recommendations.pdf

[3/84] Обработка: региональные отделения
✅ Скачан: molecular_genetic_research.pdf
✅ Скачан: региональные_отделения.pdf
✅ Скачан: temporary_recommendations.pdf

[4/84] Обработка: членство в обществе
✅ Скачан: molecular_genetic_research.pdf
✅ Скачан: членство_в_обществе.pdf
✅ Скачан: temporary_recommendations.pdf

[5/84] Обработка: заседания правления
✅ Скачан: molecular_genetic_research.pdf
✅ Скачан: заседания_правлен